# DISCURSO MAÑANERAS 

In [375]:
import pandas as pd
import re
import swifter
import numpy as np


In [376]:
def get_speech_pres(text):
    conversation = []
    for match in re.finditer(r"PRESIDENTE AND.*[\s\S]*?(?=:|\+)", text):
        match = match.group()
        match = re.sub(r"(?<=\.)[^.]*$", "", match)
        match  = re.sub(r".+: ", "", match)
        match = match.replace("\n"," ")
        conversation.append(match)
        
    return conversation

In [527]:
def identify_intervations(text):
    if '<p class="p1">' not in text:
        conversation = []
        for match in re.finditer(r".*[\s\S]*?(?=\n)", text):
            match = match.group()
            match = re.sub(r"(?<=\.)[^.]*$", "", match)
            match = match.replace("\n","")
            conversation.append(match)
        indexes = [conversation.index(match) for match in conversation if re.match(r'^[A-Z]{2,}.+[:]\s', match)]
        len_ = len(conversation)
        indexes.append(len_)
        interventions = [''.join(conversation[elem:indexes[index+1]]) for index, elem in enumerate(indexes) if elem != len_]
        return interventions
    else:
        pass

In [378]:
def get_speaker(text):
    
    text = text.split(':')[0]
    if ',' in text:
        speaker = text.split(',')[0]

        cargo = text.split(',')[1]
    else:
        speaker = text
        cargo = ''
    return speaker, cargo

In [379]:
def get_speakers(a_list):
    speakers, cargos = [], []
    try: 
        for element in a_list:
            speaker, cargo = get_speaker(element)
            speakers.append(speaker)
            cargos.append(cargo)

    except:
        pass
    speakers = ', '.join(str(item) for item in speakers)
    cargos = ', '.join(str(item) for item in cargos)
    return speakers, cargos

In [380]:
def count_words(list):
    a_list = "".join(list)
    n_word = len(a_list.split())
    return n_word

In [381]:
def n_elements(text):
    number_of_elements = len(text)
    return number_of_elements

## Preguntas de interés
1. ¿El presidente responde las preguntas de los periodistas?

Veces que responde distinto a lo vemos.

2. ¿Cuál es la demografía de los periodistas en la mañanena?
    * sexo 
    * afines al gobierno 
    * tipo de temas (desarrollar más y proponer cómo responder esto)
    
3. Cuando el presidente promete solicitar que uno de sus colaboradores asista a la mañanera para que dé respuesta a un cuestionamiento,

    * ¿asiste?
    * ¿cuánto tiempo (mañaneras depués) tarda en asistir?
    * ¿responde adecuadamente? (no se puede responder)
    
4. ¿Número de vistas al canal del presidente en cada una de sus mañaneras?
    * Proporción de las personas con acceso a internet.
5. Temas de interés:
    * ¿Para el presidente?
    * ¿Para los periodistas?
6. ¿Número de palabras del presidente por mañanera?
6. ¿Número de palabras de los periodistas por mañanera?
9. Tiempo promedio para que el primer periodista haga la primera pregunta?
    * Palabras
    * Interveciones

10. ¿Promedio de preguntas por periodistas y el promedio de preguntas por mañanera?
Promedio de intervenciones:
    + Intervenciones de periodistas por 
    + intervencuines 
11. ¿Hay invitados a la mañanera que no tomen la palabra? 
    + (cómo se responde esto)
12. Cuando el presidente pide que muestren algún material en la computadora del equipo de Jesús Cuevas ¿Cuánto tiempo pasa para que encuentre el material? (esto es un proxy de qué tan plenados son los temas)

    + contando palabras
    
    + 
    
13. ¿Cuál es la secretaria que más invitados tiene en la mañanera? ¿Los periodistas por cuál secretaria preguntan más?
14. ¿Cuál es la secretaria con mayor proporción de uso de la palabra? 
15. ¿Cuál es la secretaria a la que mayor referencia hace el presidente?

In [382]:
disco = pd.read_parquet('manianeras.parquet')

In [521]:
disco['content'][3]

'2022: Año de Ricardo Flores Magón, precursor de la Revolución Mexicana\nPRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR: Buenos días. Tu tierra, bueno, es de todos. Muchas felicidades, muchísimo gusto que amanecimos en Chetumal, en la capital de Quintana Roo, y vamos a informar, vamos a empezar con el tema de seguridad.\nHoy jueves también nos corresponde dar a conocer lo de las incidencias delictivas, los delitos que se cometen y sobre todo la forma en que se investiga y se castiga a los responsables, el Cero Impunidad. Entonces, son dos temas: información de seguridad en Quintana Roo, lo de Cero Impunidad. Posteriormente, la gobernadora Mara Lezama va a informarnos de cómo va su gobierno y al final ya abrimos para preguntas y respuestas. Esa va a ser la agenda.\nNos da muchísimo gusto estar aquí. Quintana Roo es un paraíso y lo más importante es que es un paraíso que le ha dado mucho a la nación, porque aquí a Quintana Roo han venido a buscarse la vida, a trabajar, muchos mexicanos de otros 

In [383]:
#keep only text from conferencias matutinas
disco = disco[disco['title'].str.contains("conferencia de prensa matutina del presidente")]

In [384]:
#import sys
#!{sys.executable} -m pip install swifter

In [ ]:
pip install openai

import os
import openai
openai.organization = "YOUR_ORG_ID"
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.Model.list()


response = openai.Image.create(
  prompt="a white siamese cat",
  n=1,
  size="1024x1024"
)
image_url = response['data'][0]['url']

In [385]:
#%%capture
#import IPython
#import sys
#
#!{sys.executable} -m pip install ipywidgets
#IPython.Application.instance().kernel.do_shutdown(True)  # has to restart kernel so changes are used

In [386]:
disco['date'] = disco['date'].replace(',', '', regex=True)

disco['date'] = disco.date.str.split(' ')

disco[['month', 'day', 'year']] = disco.date.apply(pd.Series)

months = {
    "enero": 1,
    "febrero": 2,
    "marzo": 3, 
    "abril": 4,
    "mayo": 5,
    "junio": 6,
    "julio": 7, 
    "agosto": 8,
    "septiembre": 9,
    "octubre": 10,
    "noviembre": 11,
    "diciembre": 12
}

disco['month'].replace(months, inplace=True)

disco['date'] = pd.to_datetime(disco[['year', 'month', 'day']])

In [387]:
disco.reset_index(drop=True, inplace=True)

In [388]:
disco['interv'] = disco['content'].apply(lambda x: identify_intervations(x))

In [389]:
disco['speakers'] = disco['interv'].apply(lambda x: get_speakers(x)[0])
disco['appointments' ] = disco['interv'].apply(lambda x: get_speakers(x)[1])

In [390]:
disco["speakers"] = disco["speakers"].str.split(",")
speakers_ = disco
speakers_ = speakers_.explode("speakers")

In [391]:
disco["appointments"] = disco["appointments"].str.split(",")
appointments_ = disco
appointments_ = appointments_.explode("appointments")

In [392]:
def remove_spaces(text):
    text = text.strip()
    return text

In [393]:
speakers_['appointments'] = appointments_['appointments'].values
speakers_ = speakers_[['date', 'speakers', 'appointments']]
speakers_['appointments'].unique()

array(['', ' ', '  SECRETARIO DE SALUD',
       '  SUBSECRETARIO DE PREVENCIÓN Y PROMOCIÓN DE LA SALUD',
       '  DIRECTOR GENERAL DEL INSTITUTO MEXICANO DEL SEGURO SOCIAL (IMSS)',
       '  BENEFICIARIA',
       '  COORDINADOR GENERAL DE COMUNICACIÓN SOCIAL Y VOCERO DE PRESIDENCIA',
       '  COORDINADOR GENERAL DE EDUCACIÓN Y DIVULGACIÓN DE LA PROCURADURÍA FEDERAL DEL CONSUMIDOR (PROFECO)',
       '  SECRETARIO DE MARINA',
       '  SUBSECRETARIO DE SEGURIDAD PÚBLICA DE LA SECRETARÍA DE SEGURIDAD Y PROTECCIÓN CIUDADANA (SSPC)',
       '  GOBERNADORA DE QUINTANA ROO', '  SECRETARIO DE GOBERNACIÓN',
       '  PRESIDENTE NACIONAL DE LA CRUZ ROJA MEXICANA',
       '  COMANDANTE DE LA GUARDIA NACIONAL',
       '  SECRETARIO DE LA DEFENSA NACIONAL',
       '  SECRETARIO DE RELACIONES EXTERIORES',
       '  PROCURADOR FEDERAL DEL CONSUMIDOR',
       '  GOBERNADOR DE LAS ISLAS MARÍAS', '  SECRETARIA DE BIENESTAR',
       '  PRESIDENTE DE FUNDACIÓN TELETÓN',
       '  MINISTRO DE DEFENSA DE 

In [394]:
speakers_['speakers'] = speakers_['speakers'].apply(lambda x: remove_spaces(x))
speakers_['appointments'] = speakers_['appointments'].apply(lambda x: remove_spaces(x))

In [395]:
len(list(set(speakers_['speakers'])))

987

In [396]:
speakers_['speakers_'] = speakers_['speakers'].apply(lambda x: remove_everything_parenthesis(x))


In [397]:
d = {
    'SECRETARIA MARÍA LUISA ALCALDE LUJÁN': 'LUISA MARÍA ALCALDE LUJÁN',
    'SECRETARIO JORGE CARLOS ALCOCER VARELA': 'JORGE CARLOS ALCOCER VARELA',
    'SECRETARIA ROCÍO NAHLE': 'ROCÍO NAHLE GARCÍA',
    'GENERAL DE BRIGADA I.C. DEM': 'RICARDO VALLEJO SUÁREZ',
    'SECRETARIO JAVIER JIMÉNEZ ESPRIÚ': 'JAVIER JIMÉNEZ ESPRIÚ',
    'SUBSECRETARIO CARLOS ALFONSO MORÁN MOGUEL': 'CARLOS ALFONSO MORÁN MOGUEL',
    'PRESIDENTE': 'ANDRÉS MANUEL LÓPEZ OBRADOR',
    'GENERAL LUIS CRESENCIO SANDOVAL': 'LUIS CRESENCIO SANDOVAL GONZÁLEZ',
    'RESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR': 'ANDRÉS MANUEL LÓPEZ OBRADOR',
    'SECRETARIO JORGE ALCOCER VARELA': 'JORGE CARLOS ALCOCER VARELA',
    'PROCURADOR ALEJANDRO GERTZ MANERO': 'ALEJANDRO GERTZ MANERO',
    'PRESIDENTE ANDRÉS MANUEL LÓPEZ OB\xadRADOR': 'ANDRÉS MANUEL LÓPEZ OBRADOR',
    'GRAL. ARTURO VELÁZQUEZ BRAVO': 'ARTURO VELÁZQUEZ BRAVO',  
    'JESÚS RAMÍREZ CUEVAS COORDINADOR GENERAL DE COMUNICACIÓN SOCIAL Y VOCERO DE LA PRESIDENCIA': 'JESÚS RAMÍREZ CUEVAS' ,
    'SECRETARIA MARÍA LUISA ALCALDE LUJÁN': 'LUISA MARÍA ALCALDE LUJÁN',
    'SECRETARIO JORGE ALCOCER': 'JORGE CARLOS ALCOCER VARELA',
    'SECRETARIA MARÍA LUISA ALBORES GONZÁLEZ': 'MARÍA LUISA ALBORES GONZÁLEZ',
    'INTERLOCUTORA (Interpretación del inglés al español)': 'INTERLOCUTORA',
    'DIRECTOR OCTAVIO ROMERO OROPEZA': 'OCTAVIO ROMERO OROPEZA',
    'SECRETARIO ALFONSO DURAZO MONTAÑO': 'ALFONSO DURAZO MONTAÑO',
    'LAURA VELÁZQUEZ ALZÚA COORDINADORA NACIONAL DE PROTECCIÓN CIVIL': 'LAURA VELÁZQUEZ ALZÚA',
    'GOBERNADOR CARLOS JOAQUÍN GONZÁLEZ': 'CARLOS JOAQUÍN GONZÁLEZ',
    'CARLOS MARTÍNEZ VELÁZQUEZ DIRECTOR GENERAL DEL INSTITUTO DEL FONDO NACIONAL DE LA VIVIENDA PARA LOS TRABAJADORES': 'CARLOS MARTÍNEZ VELÁZQUEZ',
    'SUBSECRETARIA ARIADNA MONTIEL REYES': 'ARIADNA MONTIEL REYES',
    'SUBSECRETARIO ALEJANDRO ENCINAS RODRÍGUEZ': 'ALEJANDRO ENCINAS RODRÍGUEZ',
    'SUBSECRETARIA DE BIENESTAR': 'ARIADNA MONTIEL REYES',
    'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR': 'ANDRÉS MANUEL LÓPEZ OBRADOR',
    'LUIS CRESENCIO SANDOVAL GONZÁLEZ. SECRETARIO DE LA DEFENSA NACIONAL': 'LUIS CRESENCIO SANDOVAL GONZÁLEZ',
    'PRESIDENTE ANDRÉS MANUEL LÓPE OBRADOR': 'ANDRÉS MANUEL LÓPEZ OBRADOR',
    'DIRECTOR OCTAVIO ROMERO OROPEZ': 'OCTAVIO ROMERO OROPEZA',
    'JULIO SCHERER IBARRA. CONSEJERO JURÍDICO DEL EJECUTIVO FEDERAL': 'JULIO SCHERER IBARRA',
    'ALEJANDRO ENCINAS RODRÍGUEZ. SUBSECRETARIO DE DERECHOS HUMANOS': 'ALEJANDRO ENCINAS RODRÍGUEZ',              
    'LUIS ANTONIO RAMÍREZ PINEDA DIRECTOR DEL INSTITUTO DE SEGURIDAD Y SERVICIOS SOCIALES DE LOS TRABAJADORES DEL ESTADO': 'LUIS ANTONIO RAMÍREZ PINEDA',
    'JEFA DEL SERVICIO DE ADMINISTRACIÓN TRIBUTARIA (SAT)': 'SAT',
    'SECRETARIO JOSÉ RAFAEL OJEDA DURÁN': 'JOSÉ RAFAEL OJEDA DURÁN',
    'ES ALBERTO REYES ESCAMILLA': 'ALBERTO REYES ESCAMILLA',
    'GENERAL HOMERO MENDOZA RUIZ': 'HOMERO MENDOZA RUIZ',
    'DIRECTORA MARGARITA RÍOS FARJAT': 'MARGARITA RÍOS FARJAT' ,
    'SECRETARIO MARCELO EBRARD CASAUBÓN': 'MARCELO EBRARD CASAUBÓN',         
    'BERENICE ROMERO DOMÍNGUEZ SUBPROCURADORA DE SERVICIOS Y ENCARGADA DE DESPACHO DE LA PROCURADURÍA FEDERAL DE CONSUMIDOR': 'BERENICE ROMERO DOMÍNGUEZ', 
    'ALEJANDRO ENCINAS RODRÍGUEZ SUBSECRETARIO DE DERECHOS HUMANOS': 'ALEJANDRO ENCINAS RODRÍGUEZ',
    'SECRETARIO VÍCTOR MANUEL VILLALOBOS ARÁMBULA': 'VÍCTOR MANUEL VILLALOBOS ARÁMBULA',
    'PRESIDENTE DONALD TRUMP': 'DONALD TRUMP',
    'GOBERNADOR DE NUEVO LEÓN': 'JAIME RODRÍGUEZ CALDERÓN',
    'COORDINADOR GENERAL DE COMUNICACIÓN SOCIAL Y VOCERO DE LA PRESIDENCIA': 'COORDINADOR GENERAL DE COMUNICACIÓN SOCIAL Y VOCERO DE LA PRESIDENCIA',
    'ARIADNA MONTIEL REYES. SUBSECRETARIA DE BIENESTAR': 'ARIADNA MONTIEL REYES',
    'MARCELO EBRARD CASAUBÓN SECRETARIO DE RELACIONES EXTERIORES': 'MARCELO EBRARD CASAUBÓN',
    'SUBSECRETARIO CARLOS MORÁN MOGUEL': 'CARLOS ALFONSO MORÁN MOGUEL',
    'SECRETARIA LUISA MARÍA ALCALDE': 'LUISA MARÍA ALCALDE LUJÁN',
    'PRSIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR': 'ANDRÉS MANUEL LÓPEZ OBRADOR',
    'INTERLOCUTOR Y en este caso del que hablaba el subsecretario Ricardo':'INTERLOCUTOR',
    'ADELFO REGINO': 'ADELFO REGINO MONTES',
    'ADÁN AUGUSTO LÓPEZ HERNÁDEZ': 'ADÁN AUGUSTO LÓPEZ HERNÁNDEZ',
    'ZOÉ ROBLEDO': 'ZOÉ ALEJANDRO ROBLEDO ABURTO', 
    'ZOÉ ROBLEDO ABURTO': 'ZOÉ ALEJANDRO ROBLEDO ABURTO',
    'VOCES A CORO': 'VOCES',
    'VOZ ADOLFO LÓPEZ MATEOS': 'ADOLFO LÓPEZ MATEOS',
    'VOZ HOMBRE': 'VOZ DE HOMBRE', 
    'VOZ MASCULINA': 'VOZ DE HOMBRE', 
    'VOZ MUJER': 'VOZ DE MUJER', 
    'VOZ EN OFF': 'VOCES',
    'VOZ NIÑOS': 'VOZ DE NIÑOS',
    'VOZ NIÑA Y NIÑO': 'VOZ DE NIÑOS',
    'VOZ NIÑO': 'VOZ DE NIÑOS',
    'PREFGUNTA': 'PREGUNTA',
    'PREGUNA': 'PREGUNTA', 
    'PREGUNTA INTERVENCIÓN': 'PREGUNTA',
    'PREGUNTAR': 'PREGUNTA',
    'PREGUNTAS': 'PREGUNTA',
    'PREGUNTO': 'PREGUNTA',
    'PREUGUNTA': 'PREGUNTA',
    'PRESDIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR': 'ANDRÉS MANUEL LÓPEZ OBRADOR', 
    'PRESIDENTE ALBERTO ÁNGEL FERNÁNDEZ': 'ALBERTO ÁNGEL FERNÁNDEZ', 
    'PRESIDENTE ANDRES MANUEL LÓPEZ OBRADOR': 'ANDRÉS MANUEL LÓPEZ OBRADOR', 
    'PRESIDENTE ANDRPES MANUEL LÓPEZ OBRADOR': 'ANDRÉS MANUEL LÓPEZ OBRADOR', 
    'PRESIDENTE ANDRÉS MANUEL LOPEZ OBRADOR': 'ANDRÉS MANUEL LÓPEZ OBRADOR', 
    'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOS': 'ANDRÉS MANUEL LÓPEZ OBRADOR', 
    'PRESIDENTE ANDRÉS MANUEL LÓPZ OBRADOR': 'ANDRÉS MANUEL LÓPEZ OBRADOR', 
    'PRESIDENTE ANDRÉS MANÚEL LÓPEZ OBRADOR': 'ANDRÉS MANUEL LÓPEZ OBRADOR', 
    'PRESIDENTE ANDRÉS MÁNUEL LÓPEZ OBRADOR': 'ANDRÉS MANUEL LÓPEZ OBRADOR', 
    'ALBERTO FERNÁNDEZ': 'ALBERTO HERNÁNDEZ ZAVALA',
    'ALEJADRO SVARCH PÉREZ': 'ALEJANDRO SVARCH PÉREZ',
    'ALEJANDO SVARCH PÉREZ': 'ALEJANDRO SVARCH PÉREZ',
    'ALEJANDRA FRASTO GUERRERO':'ALEJANDRA FRAUSTO GUERRERO',
    'SUBSECRETARIO ALEJANDRO ENCINAS':'ALEJANDRO ENCINAS RODRÍGUEZ',
    'ALEJANDRO ENCINA RODRÍGUEZ': 'ALEJANDRO ENCINAS RODRÍGUEZ',
    'ALEJANDRO ENCINAS RODRÍGUEZ. SUBSECRETARIO DE GOBIERNACION': 'ALEJANDRO ENCINAS RODRÍGUEZ',
    'ALFREDO DEL MARZO MAZA':'ALFREDO DEL MAZO MAZA',
    'ALICIA BÁRCENA':'ALICIA BÁRCENA IBARRA',
    'SURIT ROMERO DOMÍNGUEZ': 'SURIT BERENICE ROMERO DOMÍNGUEZ',
    'SYLVIA VARELA': 'SYLVIA LORENA VARELA RAMÓN',
    'RUTILIO ESCANDÓN CADENAS': 'RUTILIO CRUZ ESCANDÓN CADENAS',
    'MARÍA ELENA ÁLVAREZ BUYLLA': 'MARÍA ELENA ÁLVAREZ-BUYLLA ROCES',
    'MARÍA ELENA ÁLVAREZ-BUYLLLA ROCES': 'MARÍA ELENA ÁLVAREZ-BUYLLA ROCES',
    'MARÍA EUGENIA CAMPOS GALVÁNN': 'MARÍA EUGENIA CAMPOS GALVÁN',
    'MARÍA LUIS ALBORES GONZÁLEZ': 'MARÍA LUISA ALBORES GONZÁLEZ',
    'MARÍA LUISA ALBORES': 'MARÍA LUISA ALBORES GONZÁLEZ',
    'ANA ELIZABETH GARCÍA VICHIS': 'ANA ELIZABETH GARCÍA VILCHIS',
    'ANA GABRIELA GUEVARA': 'ANA GABRIELA GUEVARA ESPINOZA',
    'DIEGO PRIETO HERNÁDEZ':'DIEGO PRIETO HERNÁNDEZ',
    'DIEGO SINHUE RODRÍGUEZ VALLEJO':'DIEGO SINHUÉ RODRÍGUEZ VALLEJO',
    'ESTEBAL MOCTEZUMA BARRAGÁN': 'ESTEBAN MOCTEZUMA BARAGÁN',
    'ESTEBAN MOCTEZUMA BARRAGAN': 'ESTEBAN MOCTEZUMA BARAGÁN',
    'ESTEBAN MOCTEZUMA BARRAGÁAN': 'ESTEBAN MOCTEZUMA BARAGÁN',
    'ESTEBAN MOCTEZUMA BARRAGÁN': 'ESTEBAN MOCTEZUMA BARAGÁN',
    'ESTEBAN MOCTEZUMA BARRÁGAN': 'ESTEBAN MOCTEZUMA BARAGÁN',
    'ESTEBAN MOCTUZUMA BARRAGÁN': 'ESTEBAN MOCTEZUMA BARAGÁN',
    'EMILIO LOZOYA':'EMILIO LOZOYA AUSTIN', 
    'ALFONDO DURAZO MONTAÑO':'ALFONSO DURAZO MONTAÑO',
    'ARIANDA MONTIEL REYES': 'ARIADNA MONTIEL REYES',
    'ARTURO HERRERA':'ARTURO HERRERA GUTIÉRREZ',
    'ARTURO HERRERA GUTIÉRRREZ':'ARTURO HERRERA GUTIÉRREZ',
    'BEATRIZ GUTIÉRREZ MUELLER':'BEATRIZ GUTIÉRREZ MÜLLER',
    'CARLOS ALAZRAKI':'CARLOS ALAZRAKI GROSSMANN',
    'CARLOS MANUEL JAOQUÍN GONZÁLEZ':'CARLOS MANUEL JOAQUÍN GONZÁLEZ',
    'CIRO MURAYAMA':'CIRO MURAYAMA RENDÓN',
    'MARGARITA GONZÁLEZ SARABIA': 'MARGARITA GONZÁLEZ SARAVIA CALDERÓN',
    'MARGARITA GONZÁLEZ SARAVIA': 'MARGARITA GONZÁLEZ SARAVIA CALDERÓN',
    'MARGARITA RÍOS FARJAT': 'ANA MARGARITA RÍOS-FARJAT',
    'MARGARITA RÍOS-FAJART': 'ANA MARGARITA RÍOS-FARJAT',
    'MARGARITA RÍOS-FAJAT': 'ANA MARGARITA RÍOS-FARJAT',
    'MARGARITA RÍOS-FARJAT': 'ANA MARGARITA RÍOS-FARJAT',
    'CLAUDIA SHEINBAUM': 'CLAUDIA SHEINBAUM PARDO',
    'DAMIÁN ALCAZAR':'DAMIÁN ALCÁZAR',
    'LUIS CRESCENCIO SANDOVAL GONZÁLEZ':'LUIS CRESENCIO SANDOVAL GONZÁLEZ',
    'LUIS CRESENCIO SANDOVAL': 'LUIS CRESENCIO SANDOVAL GONZÁLEZ',
    'LUIS CRESENCIO SALDOVAL GONZÁLEZ':'LUIS CRESENCIO SANDOVAL GONZÁLEZ',
    'LUISA MARÍA ALCALDE': 'LUISA MARÍA ALCALDE LUJÁN',
    'MANUEL BARTLET DÍAZ': 'MANUEL BARTLETT DÍAZ',
    'MARCELO EBRAD CASAUBON': 'MARCELO EBRARD CASAUBÓN',
    'MARCELO EBRARD CASAUBON': 'MARCELO EBRARD CASAUBÓN',
    'MARCELO EBRARD CASAUBON SECRETARIO DE RELACIONES EXTERIORES': 'MARCELO EBRARD CASAUBÓN',
    'GERARDO PERDIGÓN': 'GERARDO PERDIGÓN VILLASEÑOR',
    'HUGO LÓPEZ GATELL': 'HUGO LÓPEZ-GATELL RAMÍREZ',
    'HUGO LÓPEZ GATELL RAMÍREZ': 'HUGO LÓPEZ-GATELL RAMÍREZ',
    'HUGO LÓPEZ-GATELL': 'HUGO LÓPEZ-GATELL RAMÍREZ',
    'HUGO LÓPEZ-GATELL RAMÍREZ.': 'HUGO LÓPEZ-GATELL RAMÍREZ',
    'HUMBERTO MOREIRA VALDEZ': 'HUMBERTO MOREIRA VALDÉS',
    'SECERETARIO ALFONSO DURAZO MONTAÑO': 'ALFONSO DURAZO MONTAÑO',
    'SECRETARIA LUISA MARÍA ALCALDE LUJÁN': 'LUISA MARÍA ALCALDE LUJÁN',
    'SECRETARIA OLGA SÁNCHEZ CORDERO': 'OLGA SÁNCHEZ CORDERO DÁVILA',
    'SECRETARIO LUIS CRESCENCIO SANDOVAL GONZÁLEZ': 'LUIS CRESENCIO SANDOVAL GONZÁLEZ',
    'SECRETARIO LUIS CRESENCIO SANDOVAL GONZÁLEZ': 'LUIS CRESENCIO SANDOVAL GONZÁLEZ',
    'SERGIO SAMANIEGO HUERTA': 'SERGIO ANTONIO SARMIENTO FERNÁNDEZ DE LARA',
    'ROSA ICELA RODRÍGUEZ': 'ROSA ICELA RODRÍGUEZ VELÁZQUEZ',
    'ROSA ICELA RODRÍGUEZ VÁZQUEZ': 'ROSA ICELA RODRÍGUEZ VELÁZQUEZ',
    'ROBERTO VELÁZQUEZ ÁLVAREZ': 'ROBERTO VELASCO ÁLVAREZ',
    'ROCÍO NAHLE': 'ROCÍO NAHLE GARCÍA',
    'ROCÍO NAHLE GARCIA': 'ROCÍO NAHLE GARCÍA',
    'RICARDO MEJIA BERDEJA': 'RICARDO SÓSTENES MEJÍA BERDEJA',
    'RICARDO MEJÍA BERDEJA':'RICARDO SÓSTENES MEJÍA BERDEJA',
    'RICARDO MEJÍA BERDEJO': 'RICARDO SÓSTENES MEJÍA BERDEJA',
    'RICARDO RODRIGUEZ VARGAS': 'RICARDO RODRÍGUEZ VARGAS',
    'RICARDO SHEFFIELD PADILLA': 'FRANCISCO RICARDO SHEFFIELD PADILLA',
    'RICARDO SHEFIELLD PADILLA': 'FRANCISCO RICARDO SHEFFIELD PADILLA',
    'RICARDO SHEFIFIELD PADILLA': 'FRANCISCO RICARDO SHEFFIELD PADILLA',
    'GOBERNADOR OMAR FAYAD MENESES': 'OMAR FAYAD MENESES',
    'HUGO ERIC FLORES CERVANTES': 'HUGO ÉRIC FLORES CERVANTES',  
    'INTERCULTOR': 'INTERLOCUTOR',
    'INTEREVENCIÓN': 'INTERVENCIÓN',
    'INTERLCUTOR': 'INTERLOCUTOR',
    'INTERLCUTORA': 'INTERLOCUTORA',
    'INTERLLOCUTOR': 'INTERLOCUTOR',
    'INTERLOCITOR': 'INTERLOCUTOR',
    'INTERLOCTOR': 'INTERLOCUTOR',
    'INTERLOCUTIR': 'INTERLOCUTOR',
    'INTERLOCUTOAR': 'INTERLOCUTORA',
    'INTELOCUTOR': 'INTERLOCUTOR',
    'INTELOCUTORA': 'INTERLOCUTORA',
    'INTERLOCUTOTRA': 'INTERLOCUTORA',
    'INTERLOCUTURA': 'INTERLOCUTORA',
    'INTERLOUTOR': 'INTERLOCUTOR',
    'INTEROLOCUTORA': 'INTERLOCUTORA',
    'INTERVECIÓN': 'INTERVENCIÓN',
    'INTERVECIÓN HOMBRE': 'INTERVENCIÓN HOMBRE',
    'INTERVENCION': 'INTERVENCIÓN',
    'INTERVENCIÓN': 'INTERVENCIÓN',
    'INTERVENCIÓN DE MUJER': 'INTERVENCIÓN MUJER',
    'INTERVENCIÓN NIÑO': 'INTERVENCIÓN NIÑO',
    'INTRVENCIÓN': 'INTERVENCIÓN',
    'IRMA ERÉNDIRA SANDOVAL':'IRMA ERÉNDIRA SANDOVAL BALLESTEROS',
    'IRMAR ERÉNDIRA SANDOVAL': 'IRMA ERÉNDIRA SANDOVAL BALLESTEROS',
    'ISRAEL VALLARTA':'ISRAEL VALLARTA CISNEROS',
    'JENARO VILLAMIL':'JENARO VILLAMIL RODRÍGUEZ',
    'JESUS RAMÍREZ CUEVAS':'ISRAEL VALLARTA CISNEROS',
    'JESÚS RAMIREZ CUEVAS': 'JESÚS RAMÍREZ CUEVAS',
    'JORGE ALCOCER': 'JORGE CARLOS ALCOCER VARELA',
    'JORGE ALCOCER VARELA': 'JORGE CARLOS ALCOCER VARELA',
    'JORGE ALCOCER VARELA SECRETARIO DE SALUD': 'JORGE CARLOS ALCOCER VARELA',
    'JOSÉ RAFAEL OJEDA DURAN':'JOSÉ RAFAEL OJEDA DURÁN',
    'JOSÉ ROSAS AISPURO': 'JOSÉ ROSAS AISPURO TORRES',
    'JUAN FRANCISCO BARRERA': 'JUAN FRANCISCO RIVERA CAVAZOS',
    'JUAN FRANCISCO BARRERA CAVAZOS': 'JUAN FRANCISCO RIVERA CAVAZOS',
    'JUAN FRANCISCO RIVERA':  'JUAN FRANCISCO RIVERA CAVAZOS',
    'JUAN RAMÓN DE LA FUENTE':'JUAN RAMÓN DE LA FUENTE RAMÍREZ',
    'JULIO CÉSAR URIAS':'JULIO CÉSAR URÍAS',
    'LAURA VELAZQUEZ ALZÚA': 'LAURA VELÁZQUEZ ALZÚA',
    'LEO ZUCKERMANN': 'LEO ZUCKERMANN BEHAR',
    'LEONARDO CURZIO': 'LEONARDO CURZIO GUTIÉRREZ',
    'LETICIA RAMIREZ AMAYA':  'LETICIA RAMÍREZ AMAYA',
    'LETICIA RAMÍREEZ AMAYA': 'LETICIA RAMÍREZ AMAYA',
    'LETICIA RAMÍREZ': 'LETICIA RAMÍREZ AMAYA',
    'LETICIA REMÍREZ AMAYA': 'LETICIA RAMÍREZ AMAYA',
    'LORENA CUELLAR CISNEROS': 'LORENA CUÉLLAR CISNEROS',
    'LORENA CUÉLLAR': 'LORENA CUÉLLAR CISNEROS',
    'LORENZO CÓRDOVA': 'LORENZO CORDOVA VIANELLO',
    'LUIS MARÍA ALCALDE LUJÁN': 'LUISA MARÍA ALCALDE LUJÁN',
    'MARATH BOLAÑOS LÓPEZ': 'MARATH BARUCH BOLAÑOS LÓPEZ',
    'MARCELO EBRARD CASAUBON (interpretación del inglés al español':'MARCELO EBRARD CASAUBÓN',
    'NADINE GASMAN ZYLBERMAN':'NADINE FLORA GASMAN ZYLBERMANN',
    'NADINE GASMAN ZYLBERMANN': 'NADINE FLORA GASMAN ZYLBERMANN',
    'NORA LUZ VÁZQUEZ': 'NORA LUZ VÁZQUEZ SÁNCHEZ',
    'NORMA NAHLE GARCÍA': 'NORMA ROCÍO NAHLE GARCÍA',
    'OCTAVIO ROMERO OROPEZ': 'OCTAVIO ROMERO OROPEZA',
    'OLGA SÁNCHEZ CORDERO': 'OLGA SÁNCHEZ CORDERO DÁVILA',
    'MARCELO EBRARD CASAUBON': 'MARCELO EBRARD CASAUBÓN'


    }
speakers_['speakers_clean'] = speakers_['speakers_'].map(d)



In [398]:
speakers_

,date,speakers,appointments,speakers_,speakers_clean
0,2022-12-28,PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR,,PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR,ANDRÉS MANUEL LÓPEZ OBRADOR
0,2022-12-28,INTERVENCIÓN,,INTERVENCIÓN,INTERVENCIÓN
0,2022-12-28,PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR,,PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR,ANDRÉS MANUEL LÓPEZ OBRADOR
0,2022-12-28,INTERVENCIÓN,,INTERVENCIÓN,INTERVENCIÓN
0,2022-12-28,PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR,,PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR,ANDRÉS MANUEL LÓPEZ OBRADOR
...,...,...,...,...,...
936,2018-12-10,PREGUNTA,,PREGUNTA,NaN
936,2018-12-10,PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR,,PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR,ANDRÉS MANUEL LÓPEZ OBRADOR
936,2018-12-10,PREGUNTA,,PREGUNTA,NaN
936,2018-12-10,PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR,,PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR,ANDRÉS MANUEL LÓPEZ OBRADOR


In [ ]:
GOBERNADOR DE NUEVO LEÓN 	JAIME RODRÍGUEZ CALDERÓN

In [399]:
def remove_everything_parenthesis(text):
    try:
        text = re.sub(r'\([^()]*\)', '', text)
        text = text.strip()
    except:
        text = ''
    return text

In [400]:
text = 'PYOTR DUBROV (interpretación del ruso al español)'

In [401]:
remove_everything_parenthesis(text)

'PYOTR DUBROV'

In [513]:
speakers_['speakers_clean'] = np.where(
    speakers_['speakers_clean'].isna(),
    speakers_['speakers_'],
    speakers_['speakers_clean']
)



In [514]:
speakers_

,date,speakers,appointments,speakers_,speakers_clean,appointment_clean,affiliation
0,2022-12-28,PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR,,PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR,ANDRÉS MANUEL LÓPEZ OBRADOR,NaN,NaN
0,2022-12-28,INTERVENCIÓN,,INTERVENCIÓN,INTERVENCIÓN,NaN,NaN
0,2022-12-28,PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR,,PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR,ANDRÉS MANUEL LÓPEZ OBRADOR,NaN,NaN
0,2022-12-28,INTERVENCIÓN,,INTERVENCIÓN,INTERVENCIÓN,NaN,NaN
0,2022-12-28,PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR,,PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR,ANDRÉS MANUEL LÓPEZ OBRADOR,NaN,NaN
...,...,...,...,...,...,...,...
936,2018-12-10,PREGUNTA,,PREGUNTA,PREGUNTA,NaN,NaN
936,2018-12-10,PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR,,PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR,ANDRÉS MANUEL LÓPEZ OBRADOR,NaN,NaN
936,2018-12-10,PREGUNTA,,PREGUNTA,PREGUNTA,NaN,NaN
936,2018-12-10,PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR,,PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR,ANDRÉS MANUEL LÓPEZ OBRADOR,NaN,NaN


In [403]:
n_manianeras_speakers = speakers_[['date', 'speakers_clean']].drop_duplicates()

In [404]:
otrs = speakers_[['date', 'speakers_clean']].drop_duplicates()

In [405]:
otrs['year_month'] = otrs['date'].dt.to_period('M')

In [406]:
otrs.groupby(['year_month', 'speakers_clean']).count()

date
year_month speakers_clean                          
2018-12                                           8
           ALEJANDRO ENCINAS RODRÍGUEZ            1
           ALEJANDRO GERTZ MANERO                 2
           ALFONSO DURAZO MONTAÑO                 1
           ANDRÉS MANUEL LÓPEZ OBRADOR            9
...                                             ...
2022-12    ROSA ISELA PALACIOS                    1
           SERGIO SARMIENTO FERNÁNDEZ DE LARA     1
           VOZ DE HOMBRE                          5
           VOZ DE MUJER                           7
           ZOÉ ALEJANDRO ROBLEDO ABURTO           2

[2342 rows x 1 columns]

In [407]:
n_manianeras_speakers = speakers_.groupby(['speakers_clean']).count().reset_index()

In [510]:
appointment = {
    'MARCELO EBRARD CASAUBÓN': 'SECRETARIO',
    'JESÚS RAMÍREZ CUEVAS': 'COORDINADOR',
    'LUIS CRESENCIO SANDOVAL GONZÁLEZ': 'SECRETARIO',
    'FRANCISCO RICARDO SHEFFIELD PADILLA': 'PROCURADOR',
    'ZOÉ ALEJANDRO ROBLEDO ABURTO': 'DIRECTOR',
    'JORGE CARLOS ALCOCER VARELA': 'SECRETARIO',
    'ROSA ICELA RODRÍGUEZ VELÁZQUEZ': 'SECRETARIA',
    'ROCÍO NAHLE GARCÍA': 'SECRETARIA',
    'OLGA SÁNCHEZ CORDERO DÁVILA': 'SECRETARIA',
    'DIEGO FERNÁNDEZ DE CEVALLOS': 'POLITICO',
    'ANA ELIZABETH GARCÍA VILCHIS': 'DIRECTORA',
    'JOSÉ RAFAEL OJEDA DURÁN': 'SECRETARIO',
    'OCTAVIO ROMERO OROPEZA': 'DIRECTOR',
    'LUISA MARÍA ALCALDE LUJÁN': 'SECRETARIA',
    'CLAUDIA SHEINBAUM PARDO': 'JEFA DE GOBIERNO',
    'ESTEBAN MOCTEZUMA BARAGÁN': 'SECRETARIO',
    'ARTURO HERRERA GUTIÉRREZ': 'SECRETARIO',
    'DAVID CERVANTES PEREDO': 'SUBSECRETARIO'
 
 }

In [518]:
affiliation = {
    'MARCELO EBRARD CASAUBÓN': 'SECRETARÍA DE RELACIONES EXTERIORES',
    'JESÚS RAMÍREZ CUEVAS': 'PRESIDENCIA',
    'LUIS CRESENCIO SANDOVAL GONZÁLEZ': 'DEFENSA NACIONAL Y MARINA',
    'FRANCISCO RICARDO SHEFFIELD PADILLA':'PROCURADURÍA FEDERAL DEL CONSUMIDOR',
    'ZOÉ ALEJANDRO ROBLEDO ABURTO':'INSTITUTO MEXICANO DEL SEGURO SOCIAL',
    'JORGE CARLOS ALCOCER VARELA': 'SECRETARÍA DE SALUD',
    'ROSA ICELA RODRÍGUEZ VELÁZQUEZ':'SECRETARÍA DE SEGURIDAD Y PROTECCIÓN CIUDADANA',
    'ROCÍO NAHLE GARCÍA': 'SECRETARÍA DE ENERGÍA',
    'OLGA SÁNCHEZ CORDERO DÁVILA': 'SECRETARÍA DE GOBERNACIÓN',
    'DIEGO FERNÁNDEZ DE CEVALLOS': 'PARTIDO ACCIÓN NACIONAL',
    'ANA ELIZABETH GARCÍA VILCHIS': 'PRESIDENCIA',
    'JOSÉ RAFAEL OJEDA DURÁN': 'SECRETARÍA DE MARIAN',
    'OCTAVIO ROMERO OROPEZA': 'PETROLEOS MEXICANOS',
    'LUISA MARÍA ALCALDE LUJÁN': 'SECRETARÍA DEL TRABAJO Y PREVISIÓN SOCIAL',
    'CLAUDIA SHEINBAUM PARDO': 'GOBIERNO DE LA CIUDAD DE MÉXICO',
    'ESTEBAN MOCTEZUMA BARAGÁN': 'SECRETARÍA DE EDUCACIÓN PÚBLICA',
    'ARTURO HERRERA GUTIÉRREZ': 'SECRETARÍA DE HACIENDA Y CRÉDITO PÚBLICO',
    'DAVID CERVANTES PEREDO':'SECRETARÍA DE DESARROLLO AGRARIO'
}

In [519]:
speakers_['appointment_clean'] = speakers_['speakers_clean'].map(appointment)
speakers_['affiliation'] = speakers_['speakers_clean'].map(affiliation)


In [526]:
list(disco[disco.date=='2022-07-04']['content'])

['2022: Año de Ricardo Flores Magón, precursor de la Revolución Mexicana\nPRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR: Buenos días. Ánimo, ánimo, ánimo.\nBueno, pues hoy vamos a conmemorar los 25 años del SAT, del Servicio de Administración Tributaria. Es la institución que procura que todos los ciudadanos contribuyan al desarrollo del país, es la que se encarga de recaudar y fortalecer la hacienda pública para que haya fondos suficientes y se pueda financiar el desarrollo del país.\nQuiero destacar dos o tres cosas:\nPrimero, que se ha cumplido con el compromiso de no aumentar impuestos, desde que llegamos no ha habido aumentos de impuestos, esto no sucedía antes. Al mismo tiempo, no se han creado impuestos nuevos.\nQuiero también dar a conocer al pueblo de México que se terminó con la práctica de la evasión fiscal y sobre todo de la condonación de los impuestos a los altos personajes de empresas, bancos, a los potentados que no pagaban impuestos, ya no hay condonación de impuestos.\nLo te

In [522]:
speakers_[speakers_['speakers_clean'] =='PAPA FRANCISCO']

,date,speakers,appointments,speakers_,speakers_clean,appointment_clean,affiliation
96,2022-07-08,PAPA FRANCISCO,,PAPA FRANCISCO,PAPA FRANCISCO,NaN,NaN
100,2022-07-04,PAPA FRANCISCO,,PAPA FRANCISCO,PAPA FRANCISCO,NaN,NaN
100,2022-07-04,PAPA FRANCISCO,,PAPA FRANCISCO,PAPA FRANCISCO,NaN,NaN
100,2022-07-04,PAPA FRANCISCO,,PAPA FRANCISCO,PAPA FRANCISCO,NaN,NaN
100,2022-07-04,PAPA FRANCISCO,,PAPA FRANCISCO,PAPA FRANCISCO,NaN,NaN
100,2022-07-04,PAPA FRANCISCO,,PAPA FRANCISCO,PAPA FRANCISCO,NaN,NaN


In [520]:
list(speakers_[~speakers_['appointment_clean'].isna()].appointment_clean.unique())

['SECRETARIO',
 'DIRECTOR',
 'COORDINADOR',
 'DIRECTORA',
 'PROCURADOR',
 'SECRETARIA',
 'JEFA DE GOBIERNO',
 'POLITICO',
 'SUBSECRETARIO']

In [499]:
30 	ALEJANDRO ENCINAS RODRÍGUEZ 	189 	189 	189 	189
42 	ALFONSO DURAZO MONTAÑO 	132 	132 	132 	132
679 	RICARDO SÓSTENES MEJÍA BERDEJA 	117 	117 	117 	117
10 	ADÁN AUGUSTO LÓPEZ HERNÁNDEZ 	98 	98 	98 	98

GRACIELA MÁRQUEZ COLÍN 	75 	75 	75 	75
199 	DELFINA GÓMEZ ÁLVAREZ 	72 	72 	72 	72
511 	LUIS RODRÍGUEZ BUCIO 	59 	59 	59 	59
471 	LAURA VELÁZQUEZ ALZÚA 	57 	57 	57 	57
341 	INTERVENCIÓN MUJER 	56 	56 	56 	56
340 	INTERVENCIÓN HOMBRE 	53 	53 	53 	53
517 	MANUEL BARTLETT DÍAZ 	53 	53 	53 	53
631 	PABLO GÓMEZ ÁLVAREZ 	50 	50 	50 	50
722 	SANTIAGO NIETO CASTILLO 	48 	48 	48 	48
103 	BLANCA JIMÉNEZ CISNEROS 	44 	44 	44 	44
74 	ARIADNA MONTIEL REYES 	44 	44 	44 	44
194 	DAVID LEÓN ROMERO 	43 	43 	43 	43
38 	ALEJANDRO SVARCH PÉREZ 	42 	42 	42 	42
646 	PEDRO ZENTENO SANTAELLA 	41 	41 	41 	41
348 	ISRAEL VALLARTA CISNEROS 	40 	40 	40 	40
205 	DIEGO PRIETO HERNÁNDEZ 	37 	37 	37 	37
32 	ALEJANDRO GERTZ MANERO 	37 	37 	37 	37
575 	MARÍA LUISA ALBORES GONZÁLEZ 	37 	37 	37 	37
634 	PABLO REINAH 	36 	36 	36 	36
247 	ERNESTO PRIETO ORTEGA 	35 	35 	35 	35
612 	NIÑO 	34 	34 	34 	34
479 	LETICIA RAMÍREZ AMAYA 	34 	34 	34 	34
611 	NIÑA 	33 	33 	33 	33
183 	CUITLÁHUAC GARCÍA JIMÉNEZ 	32 	32 	32 	32
678 	RICARDO RODRÍGUEZ VARGAS 	31 	31 	31 	31
177 	CRESCENCIO NOH CEN 	29 	29 	29 	29
236 	ENFERMERA 	26 	26 	26 	26
379 	JOAQUÍN LÓPEZ-DÓRIGA 	26
ENFERMERA 	26 	26 	26 	26
275 	FRANCISCO GARDUÑO YÁÑEZ 	26 	26 	26 	26
237 	ENRIQUE ALFARO RAMÍREZ 	25 	25 	25 	25
695 	ROGELIO JIMÉNEZ PONS 	25 	25 	25 	25
131 	CARLOS MARTÍNEZ VELÁZQUEZ 	25 	25 	25 	25
697 	ROMÁN MEYER FALCÓN 	25 	25 	25 	25
179 	CRISPINA DZUL PÉREZ 	24 	24 	24 	24
663 	RAQUEL BUENROSTRO SÁNCHEZ 	24 	24 	24 	24
323 	GUSTAVO RICARDO VALLEJO SUÁREZ 	24 	24 	24 	24
36 	ALEJANDRO MURAT HINOJOSA 	23 	23 	23 	23
659 	RAFAEL OJEDA DURÁN 	23 	23 	23 	23
356 	JAVIER JIMÉNEZ ESPRIÚ 	23 	23 	23 	23
498 	LUIS ANTONIO RAMÍREZ PINEDA 	22 	22 	22 	22
764 	VOCES 	21 	21 	21 	21
739 	SURIT BERENICE ROMERO DOMÍNGUEZ 	21 	21 	21 	21
142 	CARLOS SALAZAR LOMELÍN 	21 	21 	21 	21
392 	JORGE MENDOZA SÁNCHEZ 	21 	21 	21 	21
377 	JESÚS SEADE KURI 	20 	20 	20 	20
754 	TONY KINIL 	20 	20 	20 	20
345 	IRMA ERÉNDIRA SANDOVAL BALLESTEROS 	18 	18 	18 	18
431 	JUAN ANTONIO FERRER AGUILAR 	18 	18 	18 	18
393 	JORGE NUÑO LARA 	17 	17 	17 	17
566 	MARÍA ELENA ÁLVAREZ-BUYLLA ROCES 	17 	17 	17 	17
463 	KARLA QUINTANA OSUNA 	17 	17 	17 	17
26 	ALEJANDRA FRAUSTO GUERRERO 	16 	16 	16 	16
450 	JULIO SCHERER IBARRA 	16 	16 	16 	16
751 	TIBURCIO KUMUL 	16 	16 	16 	16
473 	LENY MARÍA DE LOS ÁNGELES CHI DZUL 	16 	16 	16 	16
MAURICIO VILA DOSAL 	15 	15 	15 	15
567 	MARÍA EUGENIA CAMPOS GALVÁN 	15 	15 	15 	15
123 	CARLOS EMILIANO CALDERÓN MERCADO 	15 	15 	15 	15
363 	JENARO VILLAMIL RODRÍGUEZ 	14 	14 	14 	14
358 	JAVIER MAY RODRÍGUEZ 	14 	14 	14 	14
711 	RUY LÓPEZ RIDAURA 	14 	14 	14 	14
696 	ROGELIO RAMÍREZ DE LA O 	14 	14 	14 	14
347 	ISIDORO PASTOR ROMÁN 	13 	13 	13 	13
45 	ALFREDO DEL MAZO MAZA 	13 	13 	13 	13
533 	MARGARITA GONZÁLEZ SARAVIA CALDERÓN 	13 	13 	13 	13
591 	MIGUEL SANTIAGO REYES HERNÁNDEZ 	13 	13 	13 	13
601 	NADINE FLORA GASMAN ZYLBERMANN 	13 	13 	13 	13
271 	FLORENCE CASSEZ 	13 	13 	13 	13
3 	ADELFO REGINO MONTES 	13 	13 	13 	13
513 	LUIZ INÁCIO LULA DA SILVA 	12 	12 	12 	12
193 	DAVID CERVANTES PEREDO 	12 	12 	12 	12
664 	RAQUEL SOSA ELÍZAGA 	12 	12 	12 	12
336 	INDIRA VIZCAÍNO SILVA 	11 	11 	11 	11
206 	DIEGO SINHUÉ RODRÍGUEZ VALLEJO 	11 	11 	11 	11
744 	TATIANA CLOUTHIER CARRILLO 	11 	11 	11 	11
24 	ALBERTO ÁNGEL FERNÁNDEZ 	11 	11 	11 	11
202 	DIANA ÁLVAREZ MAURY 	10 	10 	10 	10
615 	NORMA ROCÍO NAHLE GARCÍA 	10 	10 	10 	10
581 	MELINA VEGA 	10 	10 	10 	10
268 	FIDEL CASTRO RUZ 	10 	10 	10 	10
147 	CARLOS URZÚA MACÍAS 	10 	10 	10 	10
687 	ROBERTO VELASCO ÁLVAREZ 	10 	10 	10 	10
688 	ROCÍO MEJÍA FLORES 	10 	10 	10 	10
520 	MANUEL DE JESÚS HERNÁNDEZ GONZÁLEZ 	10 	10 	10 	10

SyntaxError: invalid syntax (1188095246.py, line 1)

In [504]:
list(speakers_[speakers_['speakers_clean']=='DAVID CERVANTES PEREDO'].appointments.unique())

['SUBSECRETARIO DE ORDENAMIENTO TERRITORIAL Y AGRARIO. SECRETARÍA DE DESARROLLO AGRARIO',
 '',
 'SUBSECRETARIO DE ORDENAMIENTO TERRITORIAL Y COMISIONADO NACIONAL DE RECONSTRUCCIÓN']

In [463]:
n_manianeras_speakers[n_manianeras_speakers['date']<11].sort_values('speakers', ascending=False).head(29)


,speakers_clean,date,speakers,appointments,speakers_
126,CARLOS LORET DE MOLA,10,10,10,10
687,ROBERTO VELASCO ÁLVAREZ,10,10,10,10
202,DIANA ÁLVAREZ MAURY,10,10,10,10
615,NORMA ROCÍO NAHLE GARCÍA,10,10,10,10
581,MELINA VEGA,10,10,10,10
147,CARLOS URZÚA MACÍAS,10,10,10,10
268,FIDEL CASTRO RUZ,10,10,10,10
520,MANUEL DE JESÚS HERNÁNDEZ GONZÁLEZ,10,10,10,10
688,ROCÍO MEJÍA FLORES,10,10,10,10
128,CARLOS MANUEL JOAQUÍN GONZÁLEZ,10,10,10,10


In [343]:
speakers_.groupby(['date', 'speakers_clean']).count()

speakers  appointments  speakers_
date       speakers_clean                                                 
2018-12-07                                      1             1          1
2018-12-10                                      2             2          2
           ANDRÉS MANUEL LÓPEZ OBRADOR         17            17         17
           NOEMÍ GUTIÉRREZ                      1             1          1
           PREGUNTA                            17            17         17
...                                           ...           ...        ...
2022-12-27 ROSA ISELA PALACIOS                  1             1          1
           VOZ DE HOMBRE                        2             2          2
           ZOÉ ALEJANDRO ROBLEDO ABURTO         3             3          3
2022-12-28 ANDRÉS MANUEL LÓPEZ OBRADOR          3             3          3
           INTERVENCIÓN                         2             2          2

[8317 rows x 3 columns]

In [339]:
speakers_[speakers_['speakers_clean']=='']

,date,speakers,appointments,speakers_clean,speakers_
1,2022-12-27,,,,
1,2022-12-27,,,,
2,2022-12-26,,,,
2,2022-12-26,,,,
4,2022-12-21,,,,
...,...,...,...,...,...
932,2018-12-19,,,,
934,2018-12-17,,,,
936,2018-12-10,,,,
936,2018-12-10,,,,


In [335]:
list(speakers_['appointments'].unique())

['',
 'SECRETARIO DE SALUD',
 'SUBSECRETARIO DE PREVENCIÓN Y PROMOCIÓN DE LA SALUD',
 'DIRECTOR GENERAL DEL INSTITUTO MEXICANO DEL SEGURO SOCIAL (IMSS)',
 'BENEFICIARIA',
 'COORDINADOR GENERAL DE COMUNICACIÓN SOCIAL Y VOCERO DE PRESIDENCIA',
 'COORDINADOR GENERAL DE EDUCACIÓN Y DIVULGACIÓN DE LA PROCURADURÍA FEDERAL DEL CONSUMIDOR (PROFECO)',
 'SECRETARIO DE MARINA',
 'SUBSECRETARIO DE SEGURIDAD PÚBLICA DE LA SECRETARÍA DE SEGURIDAD Y PROTECCIÓN CIUDADANA (SSPC)',
 'GOBERNADORA DE QUINTANA ROO',
 'SECRETARIO DE GOBERNACIÓN',
 'PRESIDENTE NACIONAL DE LA CRUZ ROJA MEXICANA',
 'COMANDANTE DE LA GUARDIA NACIONAL',
 'SECRETARIO DE LA DEFENSA NACIONAL',
 'SECRETARIO DE RELACIONES EXTERIORES',
 'PROCURADOR FEDERAL DEL CONSUMIDOR',
 'GOBERNADOR DE LAS ISLAS MARÍAS',
 'SECRETARIA DE BIENESTAR',
 'PRESIDENTE DE FUNDACIÓN TELETÓN',
 'MINISTRO DE DEFENSA DE PERÚ',
 'PERIODISTA',
 'CONSEJERO ELECTORAL',
 'SUBSECRETARIO DE SEGURIDAD PÚBLICA DE LA SECRETARÍA DE SEGURIDAD Y PROTECCIÓN CIUDADANA',
 'DI

In [330]:


print(len(sorted(speakers_['speakers_clean'].unique())))

777


In [331]:
print(len(sorted(speakers_['speakers_clean'].unique())))

777


In [332]:
sorted_spe = list(sorted(set(speakers_['speakers_clean'].unique())))
sorted_spe

['',
 'ABRAHAM VÁZQUEZ PICENO',
 'ADALBERTO PALMA GÓMEZ',
 'ADELFO REGINO MONTES',
 'ADOLFO CHÁVEZ VILLASANA',
 'ADOLFO GARCÍA-SASTRE',
 'ADOLFO LÓPEZ MATEOS',
 'ADRIANA AGUILAR ESCOBAR',
 'ADRIÁN GERARDO HERNÁNDEZ',
 'ADRIÁN GONZÁLEZ',
 'ADÁN AUGUSTO LÓPEZ HERNÁNDEZ',
 'AGUSTÍN GUSTAVO RODRÍGUEZ LÓPEZ',
 'AGUSTÍN RADILLA SUÁSTEGUI',
 'AGUSTÍN RODRÍGUEZ LÓPEZ',
 'AISHA CORONA AGUILAR',
 'ALBERTA FLORES SAUCEDO',
 'ALBERTO CASANOVA MARTÍN',
 'ALBERTO CHÁVEZ BAUTISTA',
 'ALBERTO HERNÁNDEZ ZAVALA',
 'ALBERTO HÍJAR SERRANO',
 'ALBERTO MANUEL SEPÚLVEDA COSÍO',
 'ALBERTO ORTIZ BOLAÑOS',
 'ALBERTO REYES ESCAMILLA',
 'ALBERTO VELÁZQUEZ GARCÍA',
 'ALBERTO ÁNGEL FERNÁNDEZ',
 'ALEJANDDRO ENCINAS RODRÍGUEZ',
 'ALEJANDRA FRAUSTO GUERRERO',
 'ALEJANDRO ALCOCER VARELA',
 'ALEJANDRO ANTONIO CALDERÓN ALIPI',
 'ALEJANDRO CALDERÓN ALIPI',
 'ALEJANDRO ENCINAS RODRÍGUEZ',
 'ALEJANDRO ERNESTO SVARCH PÉREZ',
 'ALEJANDRO GERTZ MANERO',
 'ALEJANDRO ISMAEL MURAT HINOJOSA',
 'ALEJANDRO MENDOZA ÁLVAREZ',
 'ALEJAN

In [336]:
speakers_[speakers_['appointments']=='JAIME RODRÍGUEZ CALDERÓN']

,date,speakers,appointments,speakers_clean,speakers_
892,2019-02-20,GOBERNADOR DE NUEVO LEÓN,JAIME RODRÍGUEZ CALDERÓN,JAIME RODRÍGUEZ CALDERÓN,GOBERNADOR DE NUEVO LEÓN


In [ ]:
len(list(set(speakers_['speakers'])))

In [ ]:
len(list(set(speakers_['appointments'])))

In [ ]:
^[A-Z]{2,}.+[:]\s

In [ ]:
import re
regex = re.compile('^[A-Z]{2,}.+[:]\s')
regex.split(string_a)